In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_hub as hub


ModuleNotFoundError: No module named 'tensorflow_hub'

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1,28*28*1).astype("float32") / 255.0
x_test = x_test.reshape(-1,  28*28*1).astype("float32") / 255.0


In [14]:
x_train.shape

(60000, 784)

In [19]:
model = keras.Sequential([
    keras.Input(shape=(28*28*1)),
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
     layers.Dense(10)
     ])

In [20]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)


model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.save("pretrained/")

Epoch 1/2
1875/1875 - 6s - loss: 2.2734 - accuracy: 0.2487 - 6s/epoch - 3ms/step
Epoch 2/2
1875/1875 - 4s - loss: 2.3026 - accuracy: 0.1022 - 4s/epoch - 2ms/step


In [22]:
# ================================================ #
#                  Pretrained-Model                #
# ================================================ #


model = keras.models.load_model("/content/pretrained")

# Freeze all model layer weights
model.trainable = False

# Can also set trainable for specific layers
# for layer in model.layers:
#  pass
    # assert should be true because of one-liner above
    # assert layer.trainable == False
    # layer.trainable = False

print(model.summary())  # for finding base input and output
base_inputs = model.layers[0].input
base_output = model.layers[-2].output
output = layers.Dense(10)(base_output)
new_model = keras.Model(base_inputs, output)

# # This model is actually identical to model we
# # loaded (this is just for demonstration and
# # and not something you would do in practice).
print(new_model.summary())

# # As usual we do compile and fit, this time on new_model
new_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

new_model.fit(x_train, y_train, batch_size=32, epochs=3, verbose=2)


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 64)                50240     
                                                                 
 dense_11 (Dense)            (None, 64)                4160      
                                                                 
 dense_12 (Dense)            (None, 64)                4160      
                                                                 
 dense_13 (Dense)            (None, 10)                650       
                                                                 
Total params: 59,210
Trainable params: 0
Non-trainable params: 59,210
_________________________________________________________________
None
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)       

In [23]:
# =================================================== #
#                Pretrained Keras Model               #
# =================================================== #

# Random data for demonstration (3 examples w. 3 classes)
x = tf.random.normal(shape=(3, 299, 299, 3))
y = tf.constant([0, 1, 2])

model = keras.applications.InceptionV3(include_top=True)
print(model.summary())

# for input you can also do model.input,
# then for base_outputs you can obviously
# choose other than simply removing the last one :)
base_inputs = model.layers[0].input
base_outputs = model.layers[-2].output
classifier = layers.Dense(3)(base_outputs)
new_model = keras.Model(inputs=base_inputs, outputs=classifier)
new_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

print(new_model.summary())
new_model.fit(x, y, epochs=15, verbose=2)


96112376/96112376 [==============================] - 0s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)           

In [24]:
# ================================================= #
#                Pretrained Hub Model               #
# ================================================= #

# Random data for demonstration (3 examples w. 3 classes)
x = tf.random.normal(shape=(3, 299, 299, 3))
y = tf.constant([0, 1, 2])

url = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4"

base_model = hub.KerasLayer(url, input_shape=(299, 299, 3))
model = keras.Sequential(
    [
        base_model,
        layers.Dense(128, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(x, y, batch_size=32, epochs=15, verbose=2)


Epoch 1/15
1/1 - 7s - loss: 2.2641 - accuracy: 0.0000e+00 - 7s/epoch - 7s/step
Epoch 2/15
1/1 - 0s - loss: 1.8365 - accuracy: 0.3333 - 40ms/epoch - 40ms/step
Epoch 3/15
1/1 - 0s - loss: 1.4955 - accuracy: 0.3333 - 39ms/epoch - 39ms/step
Epoch 4/15
1/1 - 0s - loss: 1.2568 - accuracy: 0.3333 - 41ms/epoch - 41ms/step
Epoch 5/15
1/1 - 0s - loss: 1.0645 - accuracy: 0.3333 - 39ms/epoch - 39ms/step
Epoch 6/15
1/1 - 0s - loss: 0.8959 - accuracy: 1.0000 - 38ms/epoch - 38ms/step
Epoch 7/15
1/1 - 0s - loss: 0.7716 - accuracy: 1.0000 - 30ms/epoch - 30ms/step
Epoch 8/15
1/1 - 0s - loss: 0.6853 - accuracy: 1.0000 - 28ms/epoch - 28ms/step
Epoch 9/15
1/1 - 0s - loss: 0.5928 - accuracy: 1.0000 - 29ms/epoch - 29ms/step
Epoch 10/15
1/1 - 0s - loss: 0.5119 - accuracy: 1.0000 - 28ms/epoch - 28ms/step
Epoch 11/15
1/1 - 0s - loss: 0.4471 - accuracy: 1.0000 - 29ms/epoch - 29ms/step
Epoch 12/15
1/1 - 0s - loss: 0.3868 - accuracy: 1.0000 - 28ms/epoch - 28ms/step
Epoch 13/15
1/1 - 0s - loss: 0.3323 - accuracy: 1